In [1]:
import cirq
import numpy as np
from sympy import symbols
import matplotlib.pyplot as plt
import networkx as nx
import openfermion as of
from openfermion import get_ground_state
from scipy.optimize import minimize

In [44]:
def IsingHamiltonian(G,n,w):
    A = G.edges
    O = of.QubitOperator()
    for e in A:
        O += of.QubitOperator('Z' + str(e[0]) + ' ' +'Z' + str(e[1]), 1.0)

    for x in range(n):
        O += of.QubitOperator('Z' + str(x), w[x])

    return O

def QAOAQuantumCircuit(G, n, w, p):
    A = G.edges
    qubits = cirq.LineQubit.range(n)
    qaoa = cirq.Circuit(cirq.H.on_each(qubits))
    gamma = symbols('𝛄:'+str(p))
    beta  = symbols('β:'+str(p))
    
    for x in range(p):
        for e in A:
            qaoa.append(cirq.ZZ(qubits[e[0]], qubits[e[1]])**gamma[x])

        for v in range(n):
            qaoa.append(cirq.Z(qubits[v])**(gamma[x] * w[v]))
        
        for row in qubits:
            qaoa.append(cirq.X(row)**beta[x])
    return qaoa


In [50]:
#::: Parameters
n = 4
p = 1
mu = 0.0
h = np.random.random_sample(n)
w = mu*h

#::: Graph
G = nx.cycle_graph(n)

#::: Hamiltonian
H = IsingHamiltonian(G,n,w)

#::: Quantum Circuit
qc = QAOAQuantumCircuit(G, n, w, p)

def qaoa_exp(params):
    d = {}
    for x in range(p):
        d.update({'𝛄'+str(x):params[x]})
        d.update({'β'+str(x):params[x+p]})
        
    params = cirq.ParamResolver(d)  
    results = cirq.Simulator().simulate(qc,param_resolver = params)
    
    return of.expectation(of.get_sparse_operator(H), results.final_state_vector).real


res = minimize(qaoa_exp, 
               (2*np.pi)*np.random.random_sample((1, 2*p)), 
               method='COBYLA', 
               tol=1e-10)

In [58]:
qaoa_min = res.fun
numerical_min = get_ground_state(of.get_sparse_operator(H))[0]
print('QAOA Minimum Energy:', res.fun)
print('Numerical Minimum Energy', numerical_min)
print('Ratio:', qaoa_min/numerical_min)

QAOA Minimum Energy: -1.9999998229465472
Numerical Minimum Energy -4.0
Ratio: 0.4999999557366368
